In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,47725
2,Huelva,Confirmados PDIA 14 días,944
3,Huelva,Tasa PDIA 14 días,"183,95463491630454"
4,Huelva,Confirmados PDIA 7 días,508
5,Huelva,Tasa PDIA 7 dias,"98,99253658631642"
6,Huelva,Total Confirmados,47930
7,Huelva,Curados,45904
8,Huelva,Fallecidos,423


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  47725.0


/tmp/ipykernel_3588/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13253.0


/tmp/ipykernel_3588/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3588/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3588/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3588/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 782 personas en los últimos 7 días 

Un positivo PCR cada 402 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,47725.0,508.0,944.0,513170.0,98.992537,183.954635,279.0
Huelva-Costa,28213.0,291.0,555.0,289548.0,100.501471,191.678064,168.0
Huelva (capital),13253.0,184.0,358.0,143837.0,127.922579,248.892844,110.0
Condado-Campiña,14885.0,173.0,320.0,156231.0,110.733465,204.824907,89.0
Paterna del Campo,264.0,34.0,49.0,3457.0,983.511715,1417.413943,25.0
Sierra de Huelva-Andévalo Central,4199.0,44.0,69.0,67391.0,65.290617,102.387559,22.0
Bollullos Par del Condado,1376.0,35.0,46.0,14387.0,243.275179,319.733092,22.0
Punta Umbría,1312.0,26.0,31.0,15355.0,169.325952,201.888636,16.0
Aljaraque,1703.0,25.0,47.0,21474.0,116.419857,218.869330,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granada de Río-Tinto (La),17.0,4.0,11.0,243.0,1646.090535,4526.748971,2.0
Paterna del Campo,264.0,34.0,49.0,3457.0,983.511715,1417.413943,25.0
Palos de la Frontera,1182.0,22.0,55.0,11742.0,187.361608,468.404020,5.0
Alosno,249.0,2.0,15.0,3933.0,50.851767,381.388253,1.0
Cartaya,2264.0,31.0,70.0,20083.0,154.359408,348.553503,13.0
Bollullos Par del Condado,1376.0,35.0,46.0,14387.0,243.275179,319.733092,22.0
Higuera de la Sierra,74.0,3.0,4.0,1291.0,232.378002,309.837335,2.0
Campofrío,62.0,1.0,2.0,713.0,140.252454,280.504909,0.0
Moguer,2121.0,21.0,60.0,21867.0,96.035121,274.386061,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,249.0,2.0,15.0,3933.0,50.851767,381.388253,1.0,0.133333
Gibraleón,970.0,3.0,9.0,12737.0,23.553427,70.660281,2.0,0.333333
San Juan del Puerto,773.0,1.0,3.0,9411.0,10.625863,31.877590,0.0,0.333333
Moguer,2121.0,21.0,60.0,21867.0,96.035121,274.386061,5.0,0.350000
Granada de Río-Tinto (La),17.0,4.0,11.0,243.0,1646.090535,4526.748971,2.0,0.363636
Zalamea la Real,128.0,2.0,5.0,3054.0,65.487885,163.719712,2.0,0.400000
Palos de la Frontera,1182.0,22.0,55.0,11742.0,187.361608,468.404020,5.0,0.400000
Almonte,2377.0,9.0,21.0,24507.0,36.724201,85.689803,6.0,0.428571
Palma del Condado (La),1479.0,12.0,28.0,10801.0,111.100824,259.235256,5.0,0.428571
